# Batching embeddings -- speedier?

Quick spike to work this out.

TL;DR there's very little in it at this level. Once the model's in memory both methods are about the same -- slight speed advantage to batch but nothing 

In [12]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

In [10]:
dataset = load_dataset("yelp_review_full")
len(dataset["train"]["text"])

650000

In [11]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/root/.cache/pypoetry/virtualenvs/redbox-LN_0gr02-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Batched

In [21]:
%%timeit

embeddings = model.encode(dataset["test"]["text"][:100])
embeddings[0]

21.2 s ± 2.34 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## One by one

In [22]:
%%timeit

embeddings = list(map(model.encode, dataset["test"]["text"][:100]))
embeddings[0]

26.5 s ± 4.57 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## One by one, reloading model

Trying to simulate what happens in the redis worker.

In [ ]:
%%timeit


def embed(text):
    del model
    model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
    return model.encode(text)


embeddings = list(map(embed, dataset["test"]["text"][:100]))
embeddings[0]